# Pytest的使用

测试代码有很多好处，典型地一个是它能增加我们了对自己代码的信心，并确保当我们修改代码时不会导致很多其他的问题，这能给代码维护带来很多方便。之前最开始接触python测试时，我们已经接触过unittest，unittest更简单，不过pytest更强大，日常看到的库也大多是使用pytest进行自动测试的，所以这里就简单学习下如何使用pytest。

主要参考：

- [Effective Python Testing With Pytest](https://realpython.com/pytest-python-testing/)

了解：

- pytest提供了哪些好处
- 如何确保测试是无状态的
- 如何使重复的测试更容易被理解
- 如何按名称或自定义组运行测试子集
- 如何创建和维护可重复使用的测试工具

## 如何安装pytest

使用pytest前需要安装它。与大多数Python软件包一样，可以使用pip从PyPI的虚拟环境中安装pytest。

```Shell
python -m pip install pytest
```

因为本repo下使用的是conda安装环境，所以：

```Shell
conda install -c conda-forge pytest
```

现在在安装环境中可用pytest命令了。

## 为什么要用pytest？

unittest提供了基础的测试套件，但它有一些不足之处。许多第三方测试框架试图解决 unittest 的一些问题，而 pytest 已被证明是最受欢迎的之一。pytest 是一个功能丰富、基于插件的生态系统。它的理念和功能将使测试变得更加容易，使我们的测试体验更加高效和愉快。有了pytest，普通任务需要更少的代码，高级任务可以通过各种节省时间的命令和插件来实现。它甚至可以开箱即用地运行现有的测试，包括那些用unittest编写的测试。

和大多数框架一样，有些开发模式在你刚开始使用pytest时是有意义的，但随着测试套件的增加，就会开始造成痛苦。这时候利用pytest提供的一些工具，可以使测试即使在扩展时也能保持高效和有效，后面会有关于这些内容的介绍。

### 更少的模板代码

大多数的函数测试都遵循 Arrange-Act-Assert 的模式。

1. Arrange 安排，或设置测试的条件
2. 通过调用一些函数或方法进行操作 Act
3. Assert 断言 某些结束条件为真

测试框架通常与测试断言挂钩，以便在断言失败时提供信息。例如，unittest提供了许多有用的断言工具，开箱即用。然而，即使是一套小的测试也需要相当数量的模板代码。

想象我们正在写一个测试套件，以确保 unittest 在项目中正常工作。我们想写一个总是通过的测试和一个总是失败的测试。

```Python
# test_with_unittest.py

from unittest import TestCase

class TryTesting(TestCase):
    def test_always_passes(self):
        self.assertTrue(True)

    def test_always_fails(self):
        self.assertTrue(False)
```

In [1]:
!python -m unittest discover

F.
FAIL: test_always_fails (test_with_unittest.TryTesting)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Projects\git\hydrus\7-python-project\1-use-pytest\test_with_unittest.py", line 10, in test_always_fails
    self.assertTrue(False)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 2 tests in 0.000s

FAILED (failures=1)


正如预期的那样，一个测试通过，一个失败。unittest 是有效的，但看看使用它时我们必须要做什么。

1. 从 unittest 导入 TestCase 类
2. 创建 TryTesting，一个 TestCase 的子类
3. 在 TryTesting 中为每个测试写一个方法
4. 使用 unittest.TestCase 的 self.assert* 方法之一来进行断言。

这是相当大的代码量，而且因为这是对任何测试的最低要求，我们最终会重复写同样的代码。但是如果是 pytest，我们就能直接使用Python的assert关键字来简化这个工作流程。

```Python
# test_with_pytest.py

def test_always_passes():
    assert True

def test_always_fails():
    assert False
```

看，就这么简单。不需要处理任何导入或类。因为可以使用 assert 关键字，所以也不需要学习或记住 unittest 中所有不同的 self.assert* 方法。

In [3]:
! pytest

============================= test session starts =============================
platform win32 -- Python 3.9.6, pytest-6.2.5, py-1.11.0, pluggy-1.0.0
rootdir: C:\Projects\git\hydrus\7-python-project\1-use-pytest
plugins: anyio-3.3.0
collected 4 items

test_with_pytest.py .F                                                   [ 50%]
test_with_unittest.py F.                                                 [100%]

================================== FAILURES ===================================
______________________________ test_always_fails ______________________________

    def test_always_fails():
>       assert False
E       assert False

test_with_pytest.py:7: AssertionError
________________________ TryTesting.test_always_fails _________________________

self = <test_with_unittest.TryTesting testMethod=test_always_fails>

    def test_always_fails(self):
>       self.assertTrue(False)
E       AssertionError: False is not true

test_with_unittest.py:10: AssertionError
==================

pytest显示的测试结果与unittest不同，它的测试结果会显示:

1. 系统状态，包括你所安装的 Python、pytest 和任何插件的版本
2. rootdir，即搜索配置和测试的目录
3. 运行器发现的测试的数量（可以看到刚刚写的unittest测试也执行了）

然后输出显示每个测试的状态，使用的语法与 unittest 类似。

- 点（.）表示测试通过。
- F表示测试失败。
- E表示测试引发了一个意外的异常（本例中没出现这一项）

对于失败的测试，报告给出了失败的详细情况。在上面的例子中，测试失败是因为断言False总是失败。

最后，报告给出了测试套件的整体状态报告。

下面是几个快速断言的例子。

为了在jupyter cell中运行pytest代码，这里安装了 [ipytest](https://github.com/chmp/ipytest)

In [4]:
import ipytest
ipytest.autoconfig()

In [8]:
%%ipytest

def test_uppercase():
    assert "loud noises".upper() == "LOUD NOISES"

def test_reversed():
    assert list(reversed([1, 2, 3, 4])) == [4, 3, 2, 1]

def test_some_primes():
    assert 37 in {
        num
        for num in range(1, 50)
        if num != 1 and not any([num % div == 0 for div in range(2, num)])
    }

...                                                                                          [100%]
3 passed in 0.02s


可以看到三个点，表示都运行成功了。

pytest的学习曲线比unittest要平滑，因为不需要为大多数测试学习新的结构。另外，使用assert使测试更容易理解。

## Fixtures: 状态和依赖性管理

测试经常会依赖于代码中一些对象的数据片断等。在unittest中，我们常会把这些依赖关系提取到setUp()（执行测试函数前会执行它）和tearDown()（执行测试函数后会执行它）方法中，这样类中的每个测试都可以利用它们了。但这样做时，你可能无意中使测试对某一特定数据或对象的依赖完全隐含。

随着时间的推移，隐性依赖会导致复杂的代码，使得我们必须了解这些代码才能理解写的测试代码。而测试应该帮助我们使代码更容易理解。如果测试本身都难以理解，那么我们可能就有麻烦了。

pytest采取了一种不同的方法。它引导我们进行明确的依赖性声明，由于有fixtures的存在，这些依赖性声明仍然可以重用。pytest Fixtures是为测试套件创建数据或测试替身或初始化一些系统状态的函数的。任何想使用Fixtures的测试都必须明确地接受它作为参数，所以依赖性总是在前面说明。

Fixtures也可以使用其他的Fixtures，同样通过明确声明它们的依赖关系。这意味着，随着时间的推移，你的fixtures可以变得庞大和模块化。尽管将Fixtures插入其他Fixtures的能力提供了巨大的灵活性，但随着测试套件的增长，它也会使管理依赖关系变得更具挑战性。

下面看看具体的细节。

### 何时创建Fixtures

想象正在编写一个函数 format_data_for_display()，来处理由API返回的数据。这些数据代表了一个人的列表，每个人都有一个给定的名字、姓氏和工作职位。该函数应该输出一个字符串的列表，其中包括每个人的全名（他们的给定名和他们的家庭名）、冒号和他们的头衔。为了测试这一点，可以写下面的代码。

```Python
def format_data_for_display(people):
    ...  # Implement this!

def test_format_data_for_display():
    people = [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

    assert format_data_for_display(people) == [
        "Alfonsa Ruiz: Senior Software Engineer",
        "Sayid Khan: Project Manager",
    ]
```

现在，假设需要编写另一个函数，将数据转化为逗号分隔的数值，以便在Excel中使用。这个测试看起来会非常相似。

```Python
def format_data_for_excel(people):
    ... # Implement this!

def test_format_data_for_excel():
    people = [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]

    assert format_data_for_excel(people) == """given,family,title
Alfonsa,Ruiz,Senior Software Engineer
Sayid,Khan,Project Manager
"""
```

如果自己写的几个测试都使用了相同的底层测试数据，那么我们可能会需要一个fixture。这样可以把重复的数据拉到一个单一的函数中，并用@pytest.fixture来表示该函数是一个pytest的fixture。

```Python
import pytest

@pytest.fixture
def example_people_data():
    return [
        {
            "given_name": "Alfonsa",
            "family_name": "Ruiz",
            "title": "Senior Software Engineer",
        },
        {
            "given_name": "Sayid",
            "family_name": "Khan",
            "title": "Project Manager",
        },
    ]
```

可以通过将其作为参数添加到测试中来使用该fixture。它的值将是该fixture函数的返回值。

```Python
def test_format_data_for_display(example_people_data):
    assert format_data_for_display(example_people_data) == [
        "Alfonsa Ruiz: Senior Software Engineer",
        "Sayid Khan: Project Manager",
    ]

def test_format_data_for_excel(example_people_data):
    assert format_data_for_excel(example_people_data) == """given,family,title
Alfonsa,Ruiz,Senior Software Engineer
Sayid,Khan,Project Manager
"""
```

现在每个测试都明显缩短了，但仍有一个明确的路径回到它所依赖的数据。一定要给fixture起一个具体的名字。这样，就可以在将来编写新的测试时，迅速确定是否要使用它。

### 何时应避免使用 fixtures

fixtures对于提取在多个测试中使用的数据或对象非常好。但对于需要在数据上有微小变化的测试，它们并不总是那么好。在测试套件中加入fixtures并不比加入普通数据或对象更好。它甚至可能更糟，因为增加了一层指示性的东西。

与大多数抽象概念一样，需要一些实践和思考来找到正确的fixtures使用方式。

### 规模化的 fixtures

当从测试中提取更多的fixtures时，你可能会发现一些fixtures可以从进一步的提取中受益。fixtures是模块化的，所以它们可以依赖其他的fixtures。你可能会发现，两个独立的测试模块中的fixtures有一个共同的依赖关系。在这种情况下，你能做什么？

你可以把fixtures从测试模块移到更一般的fixtures相关模块中。这样，你就可以把它们导入任何需要它们的测试模块中。当你发现自己在整个项目中反复使用一个fixtures时，这是一个好方法。

pytest在整个目录结构中寻找conftest.py模块。每个 conftest.py 都为 pytest 找到的文件树提供配置。你可以在文件的父目录和任何子目录中使用在某个特定的conftest.py中定义的任何fixtures。这是一个放置你最广泛使用的fixtures的好地方。

## Marks: 对测试进行分类

在任何大型测试套件中，有些测试不可避免地会很慢。例如，可能会有测试超时行为。不管是什么原因，当试图快速迭代一个新功能时，避免运行所有的慢速测试是好的。

pytest使能够为测试定义类别，并提供选项在运行套件时包括或排除某些类别。可以用任何数量的类别来标记一个测试。

标记测试对于按子系统或依赖关系对测试进行分类很有用。例如，如果一些测试需要访问数据库，那么你可以为它们创建一个@pytest.mark.database_access标记。

注意：因为可以给标记起任何名字，所以很容易打错或记错一个标记的名字。

pytest命令的--strict-markers标志可以确保测试中的所有标记都被注册在pytest配置中。

关于注册标记的更多信息，请查阅[pytest文档](https://docs.pytest.org/en/latest/mark.html#registering-marks)

当运行测试时，仍然可以用pytest命令默认运行它们。如果只想运行那些需要数据库访问的测试，那么可以使用pytest -m database_access。如果要运行除了那些需要数据库访问的所有测试，可以使用pytest -m "not database_access"。

pytest提供了一些开箱即用的标记。

- skip无条件地跳过一个测试。
- skipif如果传递给它的表达式评估为True，则跳过一个测试。
- xfail表示一个测试预计会失败，所以如果测试真的失败了，整个套件仍然可以获得通过的状态。
- parametrize（注意拼写）用不同的值作为参数来创建一个测试的多个变体。下面就会了解到这个标记的更多信息。

可以通过运行pytest --markers查看pytest知道的所有标记的列表。

In [9]:
! pytest --markers

@pytest.mark.anyio: mark the (coroutine function) test to be run asynchronously via anyio.

@pytest.mark.filterwarnings(warning): add a warning filter to the given test. see https://docs.pytest.org/en/stable/warnings.html#pytest-mark-filterwarnings 

@pytest.mark.skip(reason=None): skip the given test function with an optional reason. Example: skip(reason="no way of currently testing this") skips the test.

@pytest.mark.skipif(condition, ..., *, reason=...): skip the given test function if any of the conditions evaluate to True. Example: skipif(sys.platform == 'win32') skips the test if we are on the win32 platform. See https://docs.pytest.org/en/stable/reference.html#pytest-mark-skipif

@pytest.mark.xfail(condition, ..., *, reason=..., run=True, raises=None, strict=xfail_strict): mark the test function as an expected failure if any of the conditions evaluate to True. Optionally specify a reason for better reporting and run=False if you don't even want to execute the test function. If 

## Parametrization: 合并测试

前面已经看到了pytest fixtures是如何通过提取共同的依赖关系来减少代码重复的。当有几个输入和预期输出略有不同的测试时，fixtures就不那么有用了。在这种情况下，可以将一个测试定义参数化，pytest会根据指定的参数为我们创建测试的变体。

想象一下，写了一个函数来判断一个字符串是否是复数。一组初始的测试可以是这样的。

In [11]:
def is_palindrome(s):
    return s == s[::-1]

In [12]:
%%ipytest

def test_is_palindrome_empty_string():
    assert is_palindrome("")

def test_is_palindrome_single_character():
    assert is_palindrome("a")

def test_is_palindrome_mixed_casing():
    assert is_palindrome("Bob")

def test_is_palindrome_with_spaces():
    assert is_palindrome("Never odd or even")

def test_is_palindrome_with_punctuation():
    assert is_palindrome("Do geese see God?")

def test_is_palindrome_not_palindrome():
    assert not is_palindrome("abc")

def test_is_palindrome_not_quite():
    assert not is_palindrome("abab")

..FFF..                                                                                      [100%]
============================================ FAILURES =============================================
_________________________________ test_is_palindrome_mixed_casing _________________________________

    def test_is_palindrome_mixed_casing():
>       assert is_palindrome("Bob")
E       AssertionError: assert False
E        +  where False = is_palindrome('Bob')

C:\Users\hust2\AppData\Local\Temp/ipykernel_12476/1159894802.py:8: AssertionError
_________________________________ test_is_palindrome_with_spaces __________________________________

    def test_is_palindrome_with_spaces():
>       assert is_palindrome("Never odd or even")
E       AssertionError: assert False
E        +  where False = is_palindrome('Never odd or even')

C:\Users\hust2\AppData\Local\Temp/ipykernel_12476/1159894802.py:11: AssertionError
_______________________________ test_is_palindrome_with_punctuation __________

可以看到，最后两个有相同形状，其余的所有测试也有相同的形状：

```Python
def test_is_palindrome_<in some situation>():
    assert is_palindrome("<some string>")
```

这时候可以使用@pytest.mark.parametrize()在这个形状里填入不同的值，以大大减少测试代码。

In [14]:
import pytest

In [15]:
%%ipytest

@pytest.mark.parametrize("palindrome", [
    "",
    "a",
    "Bob",
    "Never odd or even",
    "Do geese see God?",
])
def test_is_palindrome(palindrome):
    assert is_palindrome(palindrome)

@pytest.mark.parametrize("non_palindrome", [
    "abc",
    "abab",
])
def test_is_palindrome_not_palindrome(non_palindrome):
    assert not is_palindrome(non_palindrome)

..FFF..                                                                                      [100%]
============================================ FAILURES =============================================
_____________________________________ test_is_palindrome[Bob] _____________________________________

palindrome = 'Bob'

    @pytest.mark.parametrize("palindrome", [
        "",
        "a",
        "Bob",
        "Never odd or even",
        "Do geese see God?",
    ])
    def test_is_palindrome(palindrome):
>       assert is_palindrome(palindrome)
E       AssertionError: assert False
E        +  where False = is_palindrome('Bob')

C:\Users\hust2\AppData\Local\Temp/ipykernel_12476/999739522.py:9: AssertionError
______________________________ test_is_palindrome[Never odd or even] ______________________________

palindrome = 'Never odd or even'

    @pytest.mark.parametrize("palindrome", [
        "",
        "a",
        "Bob",
        "Never odd or even",
        "Do geese see God?",
    

parametrize()的第一个参数是一个以逗号分隔的参数名称字符串。第二个参数是一个代表参数值的元组或列表。可以进一步参数化，以将所有测试合并成一个。

In [16]:
%%ipytest

@pytest.mark.parametrize("maybe_palindrome, expected_result", [
    ("", True),
    ("a", True),
    ("Bob", True),
    ("Never odd or even", True),
    ("Do geese see God?", True),
    ("abc", False),
    ("abab", False),
])
def test_is_palindrome(maybe_palindrome, expected_result):
    assert is_palindrome(maybe_palindrome) == expected_result

..FFF..                                                                                      [100%]
============================================ FAILURES =============================================
__________________________________ test_is_palindrome[Bob-True] ___________________________________

maybe_palindrome = 'Bob', expected_result = True

    @pytest.mark.parametrize("maybe_palindrome, expected_result", [
        ("", True),
        ("a", True),
        ("Bob", True),
        ("Never odd or even", True),
        ("Do geese see God?", True),
        ("abc", False),
        ("abab", False),
    ])
    def test_is_palindrome(maybe_palindrome, expected_result):
>       assert is_palindrome(maybe_palindrome) == expected_result
E       AssertionError: assert False == True
E        +  where False = is_palindrome('Bob')

C:\Users\hust2\AppData\Local\Temp/ipykernel_12476/3622646565.py:11: AssertionError
___________________________ test_is_palindrome[Never odd or even-True] ___________

## Durations Reports: 对抗缓慢的测试

每次从实现代码切换到测试代码的上下文时，都会产生一些开销。如果测试一开始就很慢，那么开销就会引起挫折感。

我们已经知道，当运行测试套件时，可以使用标记来过滤掉缓慢的测试。如果想提高测试速度，那么知道哪些测试可能提供最大的改进是很有用的。 pytest可以自动记录测试持续时间，并报告最重要的违规者。

使用 pytest 命令的 --durations 选项，可以在测试结果中包含一个持续时间报告。--durations期望有一个整数值n，并将报告最慢的n个测试。输出将跟随你的测试结果。例如：

```Python
$ pytest --durations=3
3.03s call     test_code.py::test_request_read_timeout
1.07s call     test_code.py::test_request_connection_timeout
0.57s call     test_code.py::test_database_read
======================== 7 passed in 10.06s ==============================
```

在持续时间报告中显示的每个测试都是一个很好的加速候选者，因为它所花费的时间超过了总测试时间的平均水平。

## 有用的 pytest 插件

有一些插件是比较常用的：

### pytest-randomly

pytest-randomly做了一件看似简单但很有价值的事情：它迫使测试以随机的顺序运行。

这是一个很好的方法，可以发现那些依赖于特定顺序运行的测试，这意味着它们对其他测试有一个有状态的依赖性。

### pytest-cov

如果想衡量测试对实现代码的覆盖程度，可能会使用覆盖率包。 pytest-cov集成了覆盖率，所以可以运行pytest --cov来查看测试覆盖率报告。

## 总结

pytest提供了一套核心的生产力功能，可以过滤和优化测试，同时还有一个灵活的插件系统，可以进一步扩展其价值。无论有一个庞大的遗留单元测试套件，还是要从头开始一个新项目，pytest都能为我们提供一些有价值的东西。